In [2]:
# 전이학습(Transfer Learning)
# - 전이 학습(Transfer Learning)이란 기존에 만들어진 모델과, 그 모델이 학습핚 가중치(weight)를 그대로
# 가져와 일부 Layer를 추가하여 학습하는 방법이다.
# - 사전에 학습이 완료된 이미지 분류에 최적화 된 가중치(weight)를 포함하는 모델을 이미 학습된 모델
# (Pre-Trained Model) 이라고 한다.
# - 우리는 이미 학습된 모델(Pre-Trained Model)을 이용하여 아주 쉽게 이미지 분류 문제를 해결핛 수 있다.
# -이미 학습된 모델 (Pre-Trained Model)
# AlexNet, VGG, GoogleNet, ResNet

# ILSVRC(이미지넷 이미지 인식 대회)
# -ILSVRC(ImageNet Large-Scale Visual Recognition Challenger)란 컴퓨터 비젂 분야의 옧림픽 으로
# 불리우는 대회이다.
# - AlexNet, VGG, GoogleNet, ResNet 등의 모델을 모두 ILSVRC 대회에서 우승 했던 모델들이다.
# - ImageNet Classification 대회에서 우승했던 모델과 그 가중치를 가져와 쓸수 있는데, 이것이 Pre-Trained
# Model 이다.

# 네트워크 깊이에 따른 error 변화
# - 2012년, 2013년 우승 모델들은 8개의 층으로 구성 되었다.
# - 2014년의 VGGNet(VGGNet 19)는 19층으로 구성되었고, GooleNet은 22층으로 구성 되었다.
# - 2015년에 이르러서는 152개 층으로 구성된 ResNet이 제안되었다.
# - 네트워크의 깊이가 깊어질수록 성능이 좋아졌음을 알 수 있다.

# VGGNet16 모델을 이용하여 치매 / 일반인 뇌 사진을 분류하는 것을 젂이 학습으로 처리해보자.

# - 데이터셋
# train - ad – 치매 학습데이터 이미지(80장)
# normal – 정상 학습데이터 이미지(80장)
# test – ad – 치매 테스트데이터 이미지(60장)
# normal - 정상 테스트데이터 이미지(60장)


In [ ]:
# 전이 학습 예제

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

# seed 설정
np.random.seed(3)
tf.compat.v1.set_random_seed(3)

# 학습 데이터를 설정
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   fill_mode='nearest')

# ./train 폴더의 학습 데이터를 불러오는 함수
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/workspace-total/workspace-python/2020_07_30_t2/train', # 학습 데이터가 저장된 위치
    target_size=(150,150),     # 이미지 크기 설정
    batch_size=5,
    class_mode='binary')      # 치매/정상 2진 분류이므로 바이너리 모드로 실행

# 테스트 데이터를 설정
test_datagen = ImageDataGenerator(rescale=1./255)

# ./test 폴더의 테스트 데이터를 불어노는 함수
test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/workspace-total/workspace-python/2020_07_30_t2/test', # 테스트 데이터가 저장된 위치
    target_size=(150,150),   # 이미지 크기 설정
    batch_size=5,
    class_mode='binary')    # 치매/정상 2진 분류이므로 바이너리 모드로 실행

# VGG16 모델 불러오기
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
transfer_model.trainable = False      # 불러온 모델을 새롭게 학습하는 것이 아니므로 false 설정
transfer_model.summary()              # 학습 구조 확인

# 모델 생성
finetune_model = models.Sequential()
finetune_model.add(transfer_model)   # 이미 학습된 모델을 추가
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(2, activation='softmax'))
finetune_model.summary()

# 모델 컴파일
finetune_model.compile(loss='sparse_categorical_crossentropy',
                       optimizer=optimizers.Adam(learning_rate=0.0002),
                       metrics=['accuracy'])

# 모델 실행
history = finetune_model.fit_generator(
    train_generator,        # train_generator를 학습 데이터셋으로 사용
    steps_per_epoch=100,
    epochs=20,
    validation_data=test_generator,   # test_generator를 테스트셋으로 사용
    validation_steps=4)

acc = history.history['accuracy']           # trainset의 정확도
val_acc = history.history['val_accuracy']   # testset의 정확도
y_vloss = history.history['val_loss']       # testset의 오차
y_loss = history.history['loss']            # trainset의 오차

# 선그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, acc, marker='.', c="red", label='Trainset_acc')
plt.plot(x_len, val_acc, marker='.', c="lightcoral", label='Testset_acc')
plt.plot(x_len, y_vloss, marker='.', c="cornflowerblue", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right') # 범례
plt.grid() # 격자 모양 출력
plt.xlabel('epoch') # x축 라벨
plt.ylabel('loss/acc') # y출 라벨
plt.show()




In [ ]:
# 전이학습

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

# seed 설정
np.random.seed(3)
tf.compat.v1.set_random_seed(3)

# 학습 데이터를 설정
train_datagen = ImageDataGenerator(rescale=1./255,
                                    horizontal_flip=True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    fill_mode='nearest')

# ./train 폴더의 학습 데이터를 불러오는 함수
train_generator = train_datagen.flow_from_directory(
 '/content/drive/My Drive/workspace-total/workspace-python/2020_07_30_t2/train',                     # 학습 데이터가 저장된 위치
 target_size=(150, 150),        # 이미지 크기 설정
 batch_size=5,
 class_mode='binary')           # 치매/정상 2진 분류이므로 바이너리 모드로 실행

# 테스트 데이터를 설정
test_datagen = ImageDataGenerator(rescale=1./255)

# ./test 폴더의 테스트 데이터를 불러오는 함수
test_generator = test_datagen.flow_from_directory(
 '/content/drive/My Drive/workspace-total/workspace-python/2020_07_30_t2/test',                      # 테스트 데이터가 저장된 위치
 target_size=(150, 150),        # 이미지 크기 설정
 batch_size=5,
 class_mode='binary')           # 치매/정상 2진 분류이므로 바이너리 모드로 실행

# VGG16 모델 불러오기
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False # 불러옦 모델을 새롭게 학습되는 것이 아니므로 false 설정
transfer_model.summary() # 학습 구조 확인

# 모델 생성
finetune_model = models.Sequential()
finetune_model.add(transfer_model) # 이미 학습된 모델을 추가
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(2, activation='softmax'))
finetune_model.summary()

# 모델 컴파일
finetune_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002),
metrics=['accuracy'])

# 모델 실행
history = finetune_model.fit_generator(
  train_generator, # train_generator를 학습 데이터셋으로 사용
  steps_per_epoch=100,
  epochs=20,
  validation_data=test_generator, # test_generator를 테스트셋으로 사용
  validation_steps=4)

acc= history.history['accuracy'] # trainset의 정확도
val_acc= history.history['val_accuracy'] # testset의 정확도
y_vloss = history.history['val_loss'] # testset의 오차
y_loss = history.history['loss'] # trainset의 오차

# 선그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, acc, marker='.', c="red", label='Trainset_acc')
plt.plot(x_len, val_acc, marker='.', c="lightcoral", label='Testset_acc')
plt.plot(x_len, y_vloss, marker='.', c="cornflowerblue", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')   # 범례
plt.grid()                      # 격자 모양 출력
plt.xlabel('epoch')             # x축 라벨
plt.ylabel('loss/acc')          # y출 라벨
plt.show()

